# Amazon CloudFront Monitoring / Analyzing

## 클라우드프론트 모니터링
- 다양한 방법으로 모니터링 가능
    - 클라우드워치 지표
    - Access Log / Real Time Log

### 클라우드프론트의 클라우드워치 지표
- 클라우드워치를 통해 다양한 지표 제공
    - 기본 지표
        - 요청 횟수/Data Transfer/4xx,5xx등 total error rate
    - 추가 지표(추가 비용)
        - Cache Hit Rage/Origin Latency/Error Rate By Status Code(401,403,404,502,503,504)
        - 추가 지표 비용은 고정. 활성화 지표 별로 한달에 한번 발생(Per Distribution)
- 지표들은 버지니아 리전(US-East-1)에서 확인

### 클라우드프론트 Access Log
- Access Log(Standard Log) : 모든 유저의 요청을 S3 버킷에 로깅. 실시간이 아니며 지연시간 발생
    - 각 배포가 속한 엣지로케이션에서 지속적으로 로그 파일을 만들고 S3 버킷으로 Flush
    - 시간 단위로 여러번 Flush
        - 최대 24시간 지연 가능
        - 아예 전송되지 않고 누락될 수 있음
        - 헤더와 쿠키의 크기가 20KB가 넘거나 URL이 8192바이트가 넘어갈 경우 클라우드프론트에서 요청을 별도로 Parse하지 않고 처리
            - 이 경우 로깅이 되지 않음
            - 바디의 크기는 상관 없음
- Real Time Log : 요청을 실시간으로 로깅. 지연 시간은 초 단위
    - 클라우드프론트의 요청 로그를 실시간으로 처리할 수 있는 기능
        - 키네시스 데이터 스트림으로 처리
            - 추후 Firehose등으로 S3에 로그 저장, Redshift / Opensearch 등으로 분석 가능
    - 추가 비용 발생
    - 로그의 지연 시간이 발생하거나 누락될 수 있음
    - 설정 값
        - Sampling Rate : 요청 중 얼마만큼의 비율을 실제로 받아볼 것인지 (1~100%)
        - Fields : 로그 내용 중 실시간으로 받아볼 필드
            - 예 : 타임스탬프, 클라이언트 IP 등등
        - Cache 동작 : 실시간 로그를 받아볼 패턴 단위의 동작(Behavior)
    - Kinesis Stream 권한 설정에 IAM 역할 필요

### 클라우드프론트 보고서 및 분석
- AWS에서 제공하는 클라우드프론트의 통계 및 분석 페이지
    - 캐시 통계, 인기 객체, 레퍼러, 사용량, 뷰어 종류 등을 배포 별로 확인 가능

### 데모 - 클라우드프론트 로깅 확인
- 클라우드프론트 + S3 Origin + OAC 프로비전 (S3 Static Hosting)
- 요청을 보내고 확인
    - 클라우드워치 지표 확인
    - 클라우드워치 실시간 로깅 설정 및 확인
        - 키네시스 데이터 스트림
        - 키네시스 파이어호스
        - 클라우드프론트 실시간 로깅 설정
        - 실제 로그 확인
- 실습 순서
1. 클라우드포메이션 템플릿 프로비전
    - 스택 생성 : 파일 템플릿 업로드
    - "AWS CloudFormation에서 IAM 리소스를 생성할 수 있음을 승인합니다" 체크
    - 스택 리소스로 이동 : DeploymentBucket : index.html 업로드
2. 클라우드프론트로 이동
    - 생성된 배포의 DNS/index.html에 접속해보자
3. S3 버킷 생성
4. 키네시스 데이터 스트림 생성 (온디맨드)
5. 파이어호스 생성
    - 소스 : 키네시스 데이터 스트림
    - 대상 : S3
    - 소스 설정 : 방금 만든 키네시스 선택
    - 대상 설정 : 로깅용 버킷 선택
    - 접두사, 오류 출력 접두사 : 스크립트 복붙
    - 시간대 : 서울
    - 버퍼힌트
        - 버퍼 크기 : 1mb
        - 버퍼 간격 : 60초
6. 배포된 사이트를 새로고침해서 요청을 남겨보자
    - 모니터링 콘솔로 이동(지표 확인)
    - 잘못된 주소로 접속 요청을 해보자(에러 레이트 증가)
7. 로그 활성화
    - S3 버킷으로 이동, 객체 소유권 활성화
    - 클라우드프론트 콘솔 : 로그로 이동
        - Log Destination을 S3 버킷으로 편집
8. 실시간 로그 확인
    - 로그 : 실시간 구성으로 이동 : 구성 생성
        - 샘플링 속도 100
        - 필드 이거저거 많이 선택
        - 엔드포인트 : 키네시스 스트림 선택
        - 새 서비스 역할 생성 선택
        - 배포 선택
9. 다시 배포 사이트로 이동해서 요청을 마구 날려보자
    - S3 버킷에 로그가 남는 것을 확인
10. 리소스 정리 : 파이어호스, 키네시스 데이터 스트림, 버킷, 클라우드포메이션 삭제

